<a href="https://colab.research.google.com/github/GoldenMAverick/Twitter_sentiment_analysis/blob/main/Twitter_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
%matplotlib inline


In [35]:
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [36]:
import nltk
nltk.download('stopwords')
print(stopwords.words('english'))


['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [37]:
column_names= ['target', 'id', 'date', 'flag', 'user', 'text']
df= pd.read_csv('/content/twitter.csv', names=column_names, encoding= 'ISO-8859-1')

In [38]:
df.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [40]:
df['target'].value_counts()

,count
target,
0,800000
4,800000


In [42]:
df.replace({'target' : {4:1}}, inplace= True)

In [43]:
df['target'].value_counts()

,count
target,
0,800000
1,800000


0 --- Negative
1 --- Positive


Stemming --- Reducing a word into its root **word**

In [45]:
port_stem= PorterStemmer()
def stemming(content):
  stemmed_content= re.sub('[^a-zA-Z]', ' ', content)
  stemmed_content= stemmed_content.lower()
  stemmed_content= stemmed_content.split()
  stemmed_content= [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content= ' '.join(stemmed_content)
  return stemmed_content

The Stemming operation will take around 45-50 mins so run the command in a simulataneous fill while doing other thing
s

In [46]:
df['stemmed_content']= df['text'].apply(stemming)

In [47]:
df.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [52]:
X= df['stemmed_content'].values
Y= df['target'].values

In [53]:
X_train, X_test, Y_train, Y_test= train_test_split(X,Y, stratify= Y, test_size=0.2, random_state=42)

In [55]:
vectorizer= TfidfVectorizer()
X_train= vectorizer.fit_transform(X_train)
X_test= vectorizer.transform(X_test)

In [56]:
print(X_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9453607 stored elements and shape (1280000, 461280)>
  Coords	Values
  (0, 307108)	0.46206048815324474
  (0, 239679)	0.15130037108228483
  (0, 146067)	0.12929728405657018
  (0, 175252)	0.224070805470346
  (0, 128605)	0.22108856600702773
  (0, 4832)	0.317074267861159
  (0, 124524)	0.18318401951949756
  (0, 205794)	0.24140229063801746
  (0, 454381)	0.20169626473577715
  (0, 286478)	0.16123218610004272
  (0, 406297)	0.2978221095272138
  (0, 220296)	0.43015677907624866
  (0, 388138)	0.20555120011808467
  (0, 154767)	0.26976607043258233
  (1, 445870)	0.6361096685891185
  (1, 161801)	0.5778049407933611
  (1, 124611)	0.5113765148324884
  (2, 125319)	0.6383069130836649
  (2, 349409)	0.22232944888223494
  (2, 444761)	0.30331529032956345
  (2, 358186)	0.19837942712286838
  (2, 267649)	0.19309660201644555
  (2, 12436)	0.2529872032123258
  (2, 453420)	0.2347069337186747
  (2, 312657)	0.3154702974657607
  :	:
  (1279997, 124611)	0.253778

In [58]:
model = LogisticRegression(max_iter=2000)
model.fit(X_train, Y_train)

LogisticRegression(max_iter=2000)

In [62]:
X_train_pred= model.predict(X_train)
train_data_accuracy= accuracy_score(Y_train, X_train_pred)
print('Accuracy score on the train data : ', train_data_accuracy)

Accuracy score on the train data :  0.7999984375


Accuracy on training data: 80%

In [63]:
X_test_pred= model.predict(X_test)
test_data_accuracy= accuracy_score(Y_test, X_test_pred)
print('Accuracy score on the test data : ', test_data_accuracy)

Accuracy score on the test data :  0.776796875


Accuracy on test data: 77.67%

**Example **

In [65]:
# Example of predicting sentiment on new data
new_tweet = "This is a great day!"

# Preprocess the new tweet
stemmed_new_tweet = stemming(new_tweet)

# Vectorize the preprocessed tweet using the same vectorizer
vectorized_new_tweet = vectorizer.transform([stemmed_new_tweet])

# Predict the sentiment
prediction = model.predict(vectorized_new_tweet)

# Print the prediction
if prediction[0] == 0:
  print("The sentiment of the tweet is Negative.")
else:
  print("The sentiment of the tweet is Positive.")

The sentiment of the tweet is Positive.


In [69]:
import pickle

In [70]:
filename= 'trained_model.sav'
pickle.dump(model,open(filename, 'wb'))

In [73]:
loaded_model= pickle.load(open('trained_model.sav', 'rb'))

In [79]:
X_new= X_test[200]

pred= loaded_model.predict(X_new)
print(pred)

if(pred[0] == 0):
  print('Negative Tweet')
else:
  print('Positive Tweet')



Positive Tweet
